## Introducción
Este cuaderno se centra en el procesamiento y análisis de un conjunto de datos de melanoma. Incluye pasos para la carga de datos, preprocesamiento, entrenamiento del modelo y evaluación.

In [87]:
# Imports
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [88]:
import torch

# Establecer el dispositivo a GPU si CUDA está disponible, de lo contrario a CPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Dispositivo configurado para usar: {device}")

# Mover el modelo a la GPU
model.to(device)

Dispositivo configurado para usar: cpu


In [89]:
# consts
cats = ["MEL","NV","BCC","AK","BKL","DF","VASC","SCC","UNK"]

train_percent = .7
val_percent = .2
test_percent = .1

## Configuración
Las siguientes celdas importan las librerías necesarias y definen constantes utilizadas a lo largo del cuaderno.

In [90]:
csv = pd.read_csv("./dataset/ISIC_2019_Training_GroundTruth.csv")

print("> Cantidad de elementos", csv.count(axis=1).size )
print("> Head de los datos del csv")
print(csv.head())

> Cantidad de elementos 25331
> Head de los datos del csv
          image  MEL   NV  BCC   AK  BKL   DF  VASC  SCC  UNK
0  ISIC_0000000  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
1  ISIC_0000001  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
2  ISIC_0000002  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
3  ISIC_0000003  0.0  1.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0
4  ISIC_0000004  1.0  0.0  0.0  0.0  0.0  0.0   0.0  0.0  0.0


In [91]:
header = list(csv.columns)
header.remove("image")

print("> Categorias")
print(header)

> Categorias
['MEL', 'NV', 'BCC', 'AK', 'BKL', 'DF', 'VASC', 'SCC', 'UNK']


In [92]:

data = pd.DataFrame({"img":[], "cat": []}, dtype=int)
rows_list = []
for entry in csv.values:
    new_row = {"img": entry[0], "cat": np.where(entry==1.0)[0][0]-1}
    rows_list.append(new_row)

data = pd.DataFrame(rows_list)


print("> Cantidad de filas", csv.count(axis=1).size)
print("> Head de los datos en el formato que usa pytorch")
print(data.head())

> Cantidad de filas 25331
> Head de los datos en el formato que usa pytorch
            img  cat
0  ISIC_0000000    1
1  ISIC_0000001    1
2  ISIC_0000002    0
3  ISIC_0000003    1
4  ISIC_0000004    0


## Carga de Datos y Exploración Inicial
El conjunto de datos se carga desde un archivo CSV, y se realiza una exploración inicial para entender su estructura y contenidos.

In [93]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

> Cantidad de imagenes de cada tipo:
[ 4522 12875  3323   867  2624   239   253   628     0]


In [94]:
# counter = list(filter(lambda elm: elm > 0, counter))
# min_cat_size = min(counter)

min_cat_size = data.cat.value_counts().min()

print("> Cantidad mínima de imagenes de un tipo:")
print(min_cat_size)

> Cantidad mínima de imagenes de un tipo:
239


In [95]:
data = pd.DataFrame(data.groupby("cat").apply(lambda cat: cat.sample(min_cat_size)).reset_index(drop=True))

print("> Cantidad de elemntos:", data.count(axis=1).size )
print("> Datos equilibrados")
print(data.head())

> Cantidad de elemntos: 1912
> Datos equilibrados
            img  cat
0  ISIC_0055930    0
1  ISIC_0054226    0
2  ISIC_0063692    0
3  ISIC_0063945    0
4  ISIC_0065697    0


In [96]:
data = data.sample(frac=1)

print("> cantidad de elementos", data.count(axis=1).size)
print("> datos barajados")
print(data.head())

> cantidad de elementos 1912
> datos barajados
               img  cat
585   ISIC_0025299    2
125   ISIC_0056271    0
1367  ISIC_0054918    5
1630  ISIC_0025707    6
1285  ISIC_0071002    5


## Preprocesamiento y Análisis de Datos
Estas celdas manejan el preprocesamiento de datos, incluyendo la limpieza, equilibrio y preparación para el aprendizaje automático.

In [97]:
counter = np.zeros(len(cats), dtype=int)

for elm in data.values:
    counter[int(elm[1])]+=1

print("> Cantidad de imagenes de cada tipo:")
print(counter)

> Cantidad de imagenes de cada tipo:
[239 239 239 239 239 239 239 239   0]


In [98]:
data.to_csv("./dataset/balanced_data.csv", index=False)

In [99]:
train_data, tmp = train_test_split(data, train_size=train_percent, stratify=data['cat'], shuffle=True)
val_data, test_data = train_test_split(tmp, test_size=test_percent/(test_percent+val_percent), stratify=tmp['cat'], shuffle=True)


print("> train", train_data.count(axis=1).size)
print(test_data.head())
print()

print("> val", val_data.count(axis=1).size)
print(val_data.head())
print()

print("> test", test_data.count(axis=1).size)
print(test_data.head())
print()

> train 1338
                           img  cat
1174              ISIC_0056654    4
366   ISIC_0015981_downsampled    1
448               ISIC_0054357    1
862               ISIC_0063250    3
519               ISIC_0070656    2

> val 382
                           img  cat
405               ISIC_0029190    1
990               ISIC_0026538    4
930               ISIC_0069120    3
1012              ISIC_0033635    4
419   ISIC_0012484_downsampled    1

> test 192
                           img  cat
1174              ISIC_0056654    4
366   ISIC_0015981_downsampled    1
448               ISIC_0054357    1
862               ISIC_0063250    3
519               ISIC_0070656    2


In [100]:
counter = np.zeros(len(cats), dtype=int)
for elm in train_data.values:
    counter[int(elm[1])]+=1

print("> train", train_data.count(axis=1).size, train_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/train_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in val_data.values:
    counter[int(elm[1])]+=1

print("> val", val_data.count(axis=1).size, val_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/val_data.count(axis=1).size})")
print()

counter = np.zeros(len(cats), dtype=int)
for elm in test_data.values:
    counter[int(elm[1])]+=1

print("> test", test_data.count(axis=1).size, test_data.count(axis=1).size/data.count(axis=1).size)
print(counter, f"({counter[0]/test_data.count(axis=1).size})")
print()

> train 1338 0.6997907949790795
[167 167 167 168 167 167 167 168   0] (0.12481315396113603)

> val 382 0.1997907949790795
[48 48 48 47 48 48 48 47  0] (0.1256544502617801)

> test 192 0.100418410041841
[24 24 24 24 24 24 24 24  0] (0.125)


## División de Datos para el Entrenamiento del Modelo
El conjunto de datos se divide en conjuntos de entrenamiento, validación y prueba para prepararse para el entrenamiento del modelo.

In [101]:
train_data.to_csv("dataset/train_data.csv", index=False)
val_data.to_csv("dataset/val_data.csv", index=False)
test_data.to_csv("dataset/test_data.csv", index=False)

# creacion de un dataset personalizado

In [102]:

from torch.utils.data import DataLoader, Dataset
image_dir = "C:/Users/elena/Desktop/universidad/3º año/FSI/pythorch/ISIC_2019_Training_Input/ISIC_2019_Training_Input"
class CustomDataset(Dataset):
    def __init__(self, dataframe, image_dir, transform=None):
        self.dataframe = dataframe
        self.image_dir = image_dir
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.dataframe.iloc[idx]['img'] + ".jpg")
        image = Image.open(img_name)
        label = self.dataframe.iloc[idx]['cat']
        if self.transform:
            image = self.transform(image)
        return image, label

In [103]:
import os
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split
from PIL import Image
import os
# Carga el conjunto de datos de entrenamiento sin ninguna normalización para calcular la media y la desviación estándar
unnormalized_transform = transforms.Compose([
    transforms.Resize((150, 150)), 
    transforms.ToTensor()
])
unnormalized_dataset = CustomDataset(data, image_dir, transform=unnormalized_transform)
loader = DataLoader(unnormalized_dataset, batch_size=64, shuffle=False)

# Calcular la media y la desviación estándar
mean_sum = torch.zeros(3)
std_sum = torch.zeros(3)
n_samples = 0

for images, _ in loader:
    images = images.view(images.size(0), images.size(1), -1)
    mean = images.mean(dim=2)
    std = images.std(dim=2)
    mean_sum += mean.sum(dim=0)
    std_sum += std.sum(dim=0)
    n_samples += images.size(0) * images.size(2)


mean = mean_sum / n_samples
std = std_sum / n_samples

# Transformaciones para el conjunto de entrenamiento
train_transform = transforms.Compose([
    transforms.Resize((150, 150)),             
    transforms.RandomRotation(180),            
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.RandomResizedCrop(size=(224, 224), scale=(0.8, 1.0), ratio=(0.75, 1.33)),
    transforms.ColorJitter(brightness=0.1, contrast=0.1),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

# Transformaciones para el conjunto de validación y test (sin data augmentation)
test_valid_transforms = transforms.Compose([
    transforms.Resize((150, 150)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize(mean=mean, std=std)
])

train_dataset = CustomDataset(train_data,image_dir, transform=train_transform)
val_dataset = CustomDataset(val_data,image_dir, transform=test_valid_transforms)
test_dataset = CustomDataset(test_data, image_dir,transform=test_valid_transforms)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)


In [104]:
from PIL import Image
from torchvision import transforms
from torch.utils.data import DataLoader, Dataset
class CustomDataset(Dataset):
    def __init__(self, dataframe, transform=None):
        self.dataframe = dataframe
        self.transform = transform

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        img_name = self.dataframe.iloc[idx]['img']
        label = self.dataframe.iloc[idx]['cat']
        
        # Aquí debes ajustar el path de acuerdo a la ubicación de tus imágenes
        image = Image.open(f'ruta/a/tus/imagenes/{img_name}.jpg')
        
        if self.transform:
            image = self.transform(image)
        return image, label

In [105]:
# imports
import torch
import torch.nn as nn
import torch.nn.functional as F
import sklearn.metrics as metrics
import matplotlib.pyplot as plt

# consts
CRITERION = nn.CrossEntropyLoss()

## Red Neuronal simple 

In [106]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        # Capas convolucionales
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3, stride=1, padding=1)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, stride=1, padding=1)
        self.conv3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3, stride=1, padding=1)

        # Max pooling
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2, padding=0)

        # Capa Dropout 
        self.dropout25 = nn.Dropout(0.25)
        self.dropout50 = nn.Dropout(0.5)

        # Capas Fully connected 
        # ajustado para 150x150
        self.fc1 = nn.Linear(in_features=128 * 18 * 18, out_features=128)
        self.fc2 = nn.Linear(in_features=128, out_features=8)

    def forward(self, x):

        x = self.pool(F.relu(self.conv1(x)))


        x = self.pool(F.relu(self.conv2(x)))
        x = self.dropout25(x)

   
        x = self.pool(F.relu(self.conv3(x)))
        x = self.dropout25(x)

       
        x = x.view(-1, 128 *18 * 18)

        x = F.relu(self.fc1(x))
        x = self.dropout50(x)

        x = self.fc2(x)
        return x


model = CNN()
model.eval()  
model

CNN(
  (conv1): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv2): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (conv3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (dropout25): Dropout(p=0.25, inplace=False)
  (dropout50): Dropout(p=0.5, inplace=False)
  (fc1): Linear(in_features=41472, out_features=128, bias=True)
  (fc2): Linear(in_features=128, out_features=8, bias=True)
)

## Preparación del Modelo
Esta sección prepara el modelo de red neuronal usando PyTorch, incluyendo la definición de la arquitectura del modelo y los cargadores de datos.

In [107]:
def plot_loss(history):
    plt.plot(history)
    plt.xlabel('Batch')
    plt.ylabel('Loss')
    plt.show()

## Funciones de Entrenamiento y Evaluación
Aquí se definen las funciones para entrenar el modelo y evaluar su rendimiento.

In [108]:
def train_loop(model, train_loader, optimizer, criterion, num_epoch, device, history = []):
    model.train()

    for epoch in range(num_epoch):

        running_loss = 0
        for i, (imgs, cats) in enumerate(train_loader, 1):
            
            imgs, cats = imgs.to(device), cats.to(device)
            optimizer.zero_grad()
            outputs = model(imgs)
            loss = criterion(outputs, cats)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()

            if not i % 10:
                print(f"Epoch {epoch+1}, Batch {i}, Loss: {running_loss/10:.4f}")
                history.append(running_loss/10)
                running_loss = 0.0
        val_loss = eval(model, train_loader, device)
        print(f"Validation Loss: {val_loss:.4f}")
        early_stopping(val_loss)

        if early_stopping.early_stop:
            print("Early stopping")
            break

    print("Finished training")

In [109]:
def eval(model, test_loader, device):
    model.eval()

    total = 0
    correct = 0

    pred = []
    real = []

    with torch.no_grad():
        for imgs, cats in test_loader:
            imgs, cats = imgs.to(device), cats.to(device)

            outputs = model(imgs)
            _, predicted = torch.max(outputs, 1)

            pred.extend(predicted.tolist())
            real.extend(cats.tolist())

            total += cats.size(0)
            correct += (predicted == cats).sum().item()

    print(f'Accuracy on test images: {100 * correct / total:.2f}%')
    print(metrics.classification_report(pred, real, target_names=dataset.classes))
    
    #return pred, real

## Utilidades Adicionales
Funciones de utilidad adicionales, como la detención temprana, se definen en esta sección.

In [110]:
class EarlyStopping:
    def __init__(self, patience=5, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_loss = None
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss < self.best_loss - self.min_delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=5, min_delta=0.01)


## Conclusión
